In [1]:
import json

# Read data

In [4]:
with open('../01-intro/documents.json', 'r') as f_in:
    raw_doc = json.load(f_in)

In [5]:
documents = []
for course_dict in raw_doc:
    course = course_dict['course']
    for doc in course_dict['documents']:
        doc['course'] = course
        documents.append(doc)

# Use Elastic search

In [6]:
from elasticsearch import Elasticsearch

In [7]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
!pip install IProgress 

In [11]:
from tqdm.auto import tqdm

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 948/948 [00:07<00:00, 134.01it/s]


In [13]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [15]:
q = "how do I run kafka?"

In [16]:
elastic_search(q)

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor

# Use OLLAMA LLM

In [18]:
from openai import OpenAI

In [19]:
clientO = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
def llm_ollama(prompt):
    response = clientO.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [21]:
llm_ollama(q)

'**Prerequisites:**\n\n* Java 1.6 or later\n* Apache Kafka client library (for brokers >= 0.10.0)\n\n**Steps:**\n\n1. **Download and set up Kafka brokers:**\n   - Download the Kafka broker binaries from the official website (kafka.apache.org).\n   - Set the `KAFKA_BOOTSTRAP_SERVERS` system property to the addresses of your Kafka brokers.\n   - Start the Kafka brokers according to the instructions provided with the installation package.\n\n2. **Create a producer object:**\n   - Use the `KafkaProducer` interface to create a new Kafka producer.\n   - Set the required properties, such as the broker address, topic name, and message properties.\n\n3. **Create a topic:**\n   - Use the `KafkaProducer` to create a new Kafka topic.\n   - The topic should be created with the specified number of partitions and replication factor.\n\n4. **Create a producer topic:**\n   - Use the `KafkaProducer` to create a new Kafka topic.\n   - The topic should be created with the specified number of partitions an

# Put together

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def rag_ollama_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_ollama(prompt)
    return answer

In [25]:
rag_ollama_es(q)

"Sure. Based on the context, the question is about how to install the necessary dependencies to run the code.\n\nThe context specifies that you should ensure that the 'dlt[duckdb]' package is installed before running the code.\n\nThe context also provides instructions on how to create a virtual environment and install packages in that environment."